In [1]:
# Project 1
# Author: Kevin Xuan

In [2]:
# import packages
import pandas as pd
from matplotlib import pyplot as plt


In [ ]:
%%bigquery sample_1

SELECT * FROM (
  SELECT
     gender, tripduration / 60 AS minutes
  FROM
    `bigquery-public-data`.new_york_citibike.citibike_trips
)
WHERE minutes < 10
LIMIT 5;

In [ ]:
# What are the 5 most popular trips that you would call "commuter trips"? 

In [5]:
%%bigquery commuter_trip

SELECT start_station_name, end_station_name, count(trip_id) as num_of_trips
FROM 
`bigquery-public-data`.san_francisco.bikeshare_trips
WHERE
(duration_sec > 5*60 and duration_sec < 2*60*60 and subscriber_type = 'Subscriber') and 
((extract(hour from start_date) >= 6 and extract(hour from end_date) <= 10) or
(extract(hour from start_date) >= 17 and extract(hour from end_date) <=20)) and
(extract(dayofweek from start_date)!= 1  and extract(dayofweek from start_date)!= 7)
GROUP BY start_station_name, end_station_name
ORDER BY count(trip_id) DESC
LIMIT 10;

Downloading: 100%|██████████| 10/10 [00:02<00:00,  3.40rows/s]


In [6]:
commuter_trip.head(10)

,start_station_name,end_station_name,num_of_trips
0,Harry Bridges Plaza (Ferry Building),2nd at Townsend,5426
1,Steuart at Market,2nd at Townsend,4768
2,San Francisco Caltrain (Townsend at 4th),Harry Bridges Plaza (Ferry Building),4573
3,San Francisco Caltrain (Townsend at 4th),Temporary Transbay Terminal (Howard at Beale),4505
4,San Francisco Caltrain (Townsend at 4th),Embarcadero at Folsom,4059
5,San Francisco Caltrain (Townsend at 4th),Steuart at Market,3786
6,Harry Bridges Plaza (Ferry Building),Embarcadero at Sansome,3730
7,2nd at Townsend,Harry Bridges Plaza (Ferry Building),3629
8,Market at 10th,San Francisco Caltrain 2 (330 Townsend),3551
9,Steuart at Market,Embarcadero at Sansome,3458


In [ ]:
'''
The 5 most popular trips that I would call "commuter trips" are from
1. Harry Bridges Plaza (Ferry Building) to 2nd at Townsend
2. Steuart at Market to 2nd at Townsend
3. San Francisco Caltrain (Townsend at 4th to Harry Bridges Plaza (Ferry Building)
4. San Francisco Caltrain (Townsend at 4th) to Temporary Transbay Terminal (Howard at Beale)
5. San Francisco Caltrain (Townsend at 4th) to Embarcadero at Folsom
'''

In [ ]:
# What are your recommendations for offers (justify based on your findings)?

In [ ]:
# 1. Count of trips under duration

In [ ]:
%%bigquery trips_duration

SELECT duration_group, count(trip_id) as trip_count
FROM
    (select trip_id,
    CASE
        WHEN duration_sec/60 > 5 and duration_sec/60 <15 then '5 - 15 min'
        WHEN duration_sec/60 >=15 and duration_sec/60 <30 then '15 - 30 min'
        WHEN duration_sec/60 >=30 and duration_sec/60 <45 then '30 - 45 min'
        WHEN duration_sec/60 >= 45 and duration_sec/60 <60 then '45 - 60 min'
        ELSE 'greater than 1 hour'
    END as duration_group
    FROM
    `bigquery-public-data`.san_francisco.bikeshare_trips
    WHERE 
    (duration_sec > 5*60 and duration_sec < 2*60*60 and subscriber_type = 'Subscriber') and 
    ((extract(hour from start_date) >= 6 and extract(hour from end_date) <= 10) or
    (extract(hour from start_date) >= 17 and extract(hour from end_date) <=20)) and
    (extract(dayofweek from start_date)!= 1  and extract(dayofweek from start_date)!= 7))
GROUP BY duration_group;

In [ ]:
# From the chart, we can see